In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

### 엑셀 파일 생성하기
파일명, 시트명, 컬럼명을 정해줍니다.

In [3]:
#엑셀파일 생성
wb = openpyxl.Workbook("41조_정의민.xlsx")        
ws = wb.create_sheet("화장품_크롤링")             
ws.append(['브랜드','상품명','카테고리','정가','할인가','아이디','별점','피부정보','피부타입','피부고민','자극도'])  #컬럼명 제공

In [4]:
######################제공하는 코드 건드리지 마세요 #############################

#크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

#브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option('detach',True)

#불필요한 에러 메세지 없애기
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


<WriteOnlyWorksheet "화장품_크롤링">

### 올리브영 스킨케어 랭킹 링크를 main_url로 두고 5위까지 제품 상세 링크를 sub_url에 저장하기

In [4]:
main_url = "https://www.oliveyoung.co.kr/store/main/getBestList.do"

response = requests.get(main_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

links = soup.select(".prd_info a") #빈칸채우기



#for반복문을 활용하여 5위까지 제품별 상세링크 sub_url에 저장
rank=1

sub_url= []

for link in links:
    raw_href = link.get('href')
    if raw_href and not raw_href.startswith('javascript:'):
        sub_url.append(raw_href)

for url in sub_url[:5]:  # 상위 5개 링크만 처리
    print(f"Rank {rank}: {url}")
    rank += 1
    time.sleep(1)


time.sleep(1)

# 1위 인덱스 0,  2위 인덱스1, 3위 인덱스2, 4위 인덱스3, 5위 인덱스4

Rank 1: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000187781&dispCatNo=90000010009&trackingCd=Best_Sellingbest
Rank 2: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000187925&dispCatNo=90000010009&trackingCd=Best_Sellingbest
Rank 3: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000188240&dispCatNo=90000010009&trackingCd=Best_Sellingbest
Rank 4: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000162249&dispCatNo=90000010009&trackingCd=Best_Sellingbest
Rank 5: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000185132&dispCatNo=90000010009&trackingCd=Best_Sellingbest


### 크롤링 함수 만들기
제품 상세 페이지에서 상단 제품정보(브랜드명,상품명,카테고리,정가,할인가)와  
하단의 리뷰고객 좌픅정보(아이디,별점,피부정보)와 우측정보(피부타입,피부고민,자극도)를 가져옵니다.  
위의 데이터를 엑셀에 계속 추가하는 함수를 만듭니다. 

In [4]:
main_url = f'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000187781&dispCatNo=90000010009&trackingCd=Best_Sellingbest'

response = requests.get(main_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

links = soup.select(".prd_info a") #빈칸채우기

In [5]:
soup.select_one(".pd_arrow_link").get_text().strip()

'구달'

In [7]:
soup.select_one(".prd_name").get_text().strip()

'[홍현희 PICK/ 한정기획] 구달 청귤 비타C 잡티케어 세럼 50ml+50ml 리필기획'

In [8]:
soup.select_one("#dtlCatNm").get_text().strip()

'에센스/세럼'

In [9]:
soup.select_one(".price-1 > strike").get_text().strip()

'43,000'

In [10]:
soup.select_one(".price-2 > strong").get_text().strip()

'28,900'

In [ ]:
//*[@id="gdasList"]/li[1]/div[2]   #첫번째 리뷰 전체
//*[@id="gdasList"]/li[2]/div[2]   #두번째 리뷰 전체

## 리뷰를 따오는걸 위에같이 전체를 따와서 엑셀로 만든다음에 잘라버릴까?
## 아니면 원하는 부분 XPATH만 따와서 병합을 시킬까?
## 일단 코드는 전체 따오는거로 만들기는 했음

In [ ]:
review_list_wrap = driver.find_element(By.CSS_SELECTOR,"gdasList > li")

In [ ]:
gdasList > li
//*[@id=\"gdasList\"]/li[2]
//*[@id="gdasList"]/li[1]

In [11]:
from selenium.webdriver.common.by import By

Options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
Options.add_argument('user-agent='+user_agent)
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),options=Options)

url = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000187781&dispCatNo=90000010009&trackingCd=Best_Sellingbest"
driver.get(url)
# headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

driver.find_element(By.CLASS_NAME,'goods_reputation').click()
time.sleep(1)
review_list_wrap = driver.find_element(By.XPATH,"//*[@id=\"gdasList\"]/li[2]")


    
print(review_list_wrap.text)

# 웹 드라이버 종료
driver.quit()

뚜딘
TOP 885
복합성 웜톤 다크서클 모공
한달이상사용
5점만점에 5점
2023.07.15 매장
피부타입
복합성에 좋아요
피부고민
주름/미백에 좋아요
자극도
자극없이 순해요
잡티세럼 한달 사용 후기 남깁니다❤️

저는 아이소이 잡티세럼도 사용해 보았기에 많은 분들이 저처럼 잡티세럼 구매 전 어떤 제품을 살지 고민하지 않으실까 하여 비교후기 남겨볼게용

❣️아이소이 VS 구달❣️

💌 자극정도
아이소이 > 구달

💌 잡티 케어 효과
아이소이 = 구달

💌 발림성
구달 > 아이소이

💌 발림성
아이소이 > 구달

💌 향(개인적인 의견)
구달 > 아이소이

전 참고로 향 강한건 싫어해용..
그나마 청귤향이 적응도 쉽고 더 상큼했어용

❣️ 그래서 한달 사용 결과 잡티케어 효과는?
솔직히 구달이나 아이소이나
큰 피부의 변화는 못느꼈어요

잡티케어도 되는건지 음 솔직히 큰 변화 모르겠고,
다만 바르고 나면 피부가 환해지고 결이 좀 개선된 느낌 정도는 있어용

확연한 효과를 가져다주는지는 사실 의문이에용
그냥 이미 기미와 여드름 흉터가 가득한 피부를 바꿔준다기 보단 예방에 가깝지 않을까 싶어요
구달 잡티 미백 주름 청귤 세럼 잡티세럼
※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.
이 리뷰가 도움이 돼요! 34
신고하기


In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

Options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
Options.add_argument('user-agent=' + user_agent)

# Chrome 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options)

url = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000187781&dispCatNo=90000010009&trackingCd=Best_Sellingbest"
driver.get(url)

driver.find_element(By.CLASS_NAME, 'goods_reputation').click()

time.sleep(1)

for i in range(1, 101):
    xpath = f"//*[@id=\"gdasList\"]/li[{i}]"
    try:
        review_list_wrap = driver.find_element(By.XPATH, xpath)
        print(f"Review {i}:", review_list_wrap.text)
        print("-" * 50)
    except:
        print(f"Review {i} not found")
        print("-" * 50)

driver.quit()


Review 1: 고먐미먼지
복합성 겨울쿨톤 미백 블랙헤드
5점만점에 3점
2023.08.01
피부타입
지성에 좋아요
피부고민
주름/미백에 좋아요
자극도
자극없이 순해요
이거 하나만 바르기엔 건성분들이나 복합성 수부지 분들은 시간이 지나면 건조하다 느낄거같음 그리고 좀 많이바르면 거품?처럼 흰색이 생기는데 뭔진모르겠지만 흡수시키면 ㄱㅊ아짐 솔직히 미백 잡티이딴거 효과 모르겠음...거의 비워가는데 그냥 이거살바에 다른거 사세요
※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.
이 리뷰가 도움이 돼요! 33
신고하기
--------------------------------------------------
Review 2: 뚜딘
TOP 885
복합성 웜톤 다크서클 모공
한달이상사용
5점만점에 5점
2023.07.15 매장
피부타입
복합성에 좋아요
피부고민
주름/미백에 좋아요
자극도
자극없이 순해요
잡티세럼 한달 사용 후기 남깁니다❤️

저는 아이소이 잡티세럼도 사용해 보았기에 많은 분들이 저처럼 잡티세럼 구매 전 어떤 제품을 살지 고민하지 않으실까 하여 비교후기 남겨볼게용

❣️아이소이 VS 구달❣️

💌 자극정도
아이소이 > 구달

💌 잡티 케어 효과
아이소이 = 구달

💌 발림성
구달 > 아이소이

💌 발림성
아이소이 > 구달

💌 향(개인적인 의견)
구달 > 아이소이

전 참고로 향 강한건 싫어해용..
그나마 청귤향이 적응도 쉽고 더 상큼했어용

❣️ 그래서 한달 사용 결과 잡티케어 효과는?
솔직히 구달이나 아이소이나
큰 피부의 변화는 못느꼈어요

잡티케어도 되는건지 음 솔직히 큰 변화 모르겠고,
다만 바르고 나면 피부가 환해지고 결이 좀 개선된 느낌 정도는 있어용

확연한 효과를 가져다주는지는 사실 의문이에용
그냥 이미 기미와 여드름 흉터가 가득한 피부를 바꿔준다기 보단 예방에 가깝지 않을까 싶어요
구달 잡티 미백 주름 청귤 세럼 잡티세

Review 62 not found
--------------------------------------------------
Review 63 not found
--------------------------------------------------
Review 64 not found
--------------------------------------------------
Review 65 not found
--------------------------------------------------
Review 66 not found
--------------------------------------------------
Review 67 not found
--------------------------------------------------
Review 68 not found
--------------------------------------------------
Review 69 not found
--------------------------------------------------
Review 70 not found
--------------------------------------------------
Review 71 not found
--------------------------------------------------
Review 72 not found
--------------------------------------------------
Review 73 not found
--------------------------------------------------
Review 74 not found
--------------------------------------------------
Review 75 not found
--------------------------------------------------
Review

In [ ]:
from selenium import webdriver

# Selenium 웹 드라이버 초기화
driver = webdriver.Chrome()  # 웹 드라이버 설정에 맞게 변경

# 웹 페이지 열기
url = 'https://www.example.com'  # 원하는 웹 페이지의 URL로 변경
driver.get(url)

# 원하는 요소를 찾아 텍스트 추출
try:
    review_list_wrap = driver.find_element_by_class_name("review_list_wrap")
    review_list_text = review_list_wrap.text.strip()
except:
    review_list_text = "없음"

print(review_list_text)

# 웹 드라이버 종료
driver.quit()

In [42]:
def customer_info(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    for j in range(0,10,1):

        #브랜드
        try:
            brand = soup.select_one(".pd_arrow_link").get_text().strip()
        except:
            brand ="없음"

        #상품명
        try:
            p_name = soup.select_one(".prd_name").get_text().strip()
        except:
            p_name="없음"

        #카테고리
        try:
            p_category = soup.select_one("#dtlCatNm").get_text().strip()
        except:
            p_category="없음"

        #정가
        try:
            price = soup.select_one(".price-1 > strike").get_text().strip()
        except:
            price=0

        #할인가
        try:
            discount = soup.select_one(".price-2 > strong").get_text().strip()
        except:
            discount=0
    
        driver.get(url)
        #고객 아이디
        try:
            _id = 
        except:
            _id ="없음"

        #별점
        try:
            _star = 
        except:
            _star="없음"

        #고객 피부 정보
        try:
            _info = 
        except:
            _info = "없음"


        #피부 타입
        try:
            skin_type = 
        except:
            skin_type = "없음"
            
        #피부 고민
        try:
            skin_trouble = 
        except:
            skin_trouble = "없음"
        #자극도
        try:
            skin_irritation = 
        except:
            skin_irritation = "없음"
            
        #엑셀 데이터 쌓기
        ws.append([brand,p_name,p_category,price,discount,_id, _star, _info, skin_type, skin_trouble, skin_irritation])
        time.sleep(1)


In [ ]:
#여기는 원본
def customer_info():

    for j in range(0,10,1):

        #브랜드
        try:
            brand = 
        except:
            brand ="없음"

        #상품명
        try:
            p_name =
        except:
            p_name="없음"

        #카테고리
        try:
            p_category = 
        except:
            p_category="없음"

        #정가
        try:
            price =
        except:
            price=0

        #할인가
        try:
            discount =
        except:
            discount=0

        #고객 아이디
        try:
            _id = 
        except:
            _id ="없음"

        #별점
        try:
            _star = 
        except:
            _star="없음"

        #고객 피부 정보
        try:
            _info = 
        except:
            _info = "없음"


        #피부 타입
        try:
            skin_type = 
        except:
            skin_type = "없음"
            
        #피부 고민
        try:
            skin_trouble = 
        except:
            skin_trouble = "없음"
        #자극도
        try:
            skin_irritation = 
        except:
            skin_irritation = "없음"
            
        #엑셀 데이터 쌓기
        ws.append([brand,p_name,p_category,price,discount,_id, _star, _info, skin_type, skin_trouble, skin_irritation])
        time.sleep(1)


### 웹페이지 해당 주소로 이동하여 크롤링함수를 실행하는 반복문을 만듭니다.
반복문에 들어가야 하는 것  
1. 리뷰버튼 클릭  
2. 화면 맨 아래까지 스크롤하기 (코드제공)
3. 10페이지 이하일 때 : 10페이지 크롤링 한 후 다음페이지 화살표 버튼 누르기 , 마지막 페이지인 경우 last_page==True
4. 11페이지부터 규칙을찾아 다음페이지로 이동하게 만들기, 마지막 페이지인 경우 last_page==True

In [ ]:
#웹페이지 해당 주소 이동
for i in range(0,5):          #전체 제품을 한번에 크롤링하지 않고 나눠서 크롤링 할 경우 sub_url 인덱스 고려해서 range숫자 변경


    driver.implicitly_wait(5)  #웹페이지 로딩 될때까지 5초는 기다림
    driver.maximize_window()   #화면 최대화
    driver.get(sub_url[i])          
    time.sleep(3)


    #리뷰버튼 클릭
    rv = 
    rv.click()
    time.sleep(3)
    

    #리뷰 하단 끝까지 스크롤하는 함수 (빈칸없음.그대로 코드 사용가능)
    before_h = driver.execute_script("return window.scrollY")         #스크롤 전 높이
    #화면 맨아래까지 스크롤
    while True:
        driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)

        time.sleep(1)

        #스크롤 후 높이
        after_h = driver.execute_script("return window.scrollY")

        #스크롤 값이 같으면 스크롤 멈춤
        if after_h == before_h:
            break
        before_h = after_h   

        

        
        
    last_page=False
    
    for k in range(1,101):  #100페이지 크롤링 한다고 했을 때 (상품당 최대 100페이지까지 있음)

        #마지막 페이지면 종료
        if last_page == True:
            break

        #페이지 숫자 10이하 일 때
        if k<11:
            try:    
                if k != 10:

                    

                elif k == 10:          #10페이지 크롤링 한 후에 다음페이지로 가는 화살표 버튼 클릭

                    
            except:
                   
                    

       #페이지 숫자 11이상 일 때  (규칙을 찾아 각 페이지 크롤링 후 다음 페이지로 이동하도록 코드 작성)        
        elif k>10 :

            try:



            except:
                   

                

### 크롤링한 결과를 엑셀에 저장 (상단에서 만든 엑셀 파일명과 동일하게)

In [ ]:
wb.save("파일명.xlsx")       